In [1]:
import librosa, librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
import math
import pandas as pd

In [2]:
df = pd.read_csv('MLNS_with_mfcc_stats_05242024.csv')
df=df.drop(33)

/var/folders/5z/7fkzpcz93gz6mqgh9vjgfymm0000gn/T/ipykernel_61983/1637493344.py:1: DtypeWarning: Columns (26,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MLNS_with_mfcc_stats_05242024.csv')


In [15]:
df['critter_name'].value_counts()

critter_name
cricket    3976
kaydid     1926
cicada       57
Name: count, dtype: int64

In [21]:
df.loc[df['critter_name']=='cricket', 'critter_num'] = 0
df.loc[df['critter_name']=='kaydid', 'critter_num'] = 1
df.loc[df['critter_name']=='cicada', 'critter_num'] = 2

In [22]:
df['critter_num'].value_counts()

critter_num
0.0    3976
1.0    1926
2.0      57
Name: count, dtype: int64

In [83]:
df_bin = df.drop(df[df.critter_num == 2].index)

In [23]:
from sklearn.model_selection import train_test_split

In [53]:
def truncate_mfcc(df, avg_depth=20, var_depth=10):
    df_temp=pd.DataFrame
    for n in range(0,avg_depth):
        df_temp=pd.concat([df_temp, df['mfcc_'+str(n)+'_avg']], axis=1)
    for n in range(0,var_depth):
        df_temp=pd.concat([df_temp, df['mfcc_'+str(n)+'_var']], axis=1)        
    return df_temp

In [72]:
X=truncate_mfcc(df, avg_depth=40, var_depth=40)
Y = df['critter_num']

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X.copy(), Y,
                                                       shuffle=True,
                                                       random_state=413,
                                                       test_size=.2,
                                                       stratify=Y)

In [74]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [75]:
## Make LDA object
LDA = LinearDiscriminantAnalysis()

## Fit the model
LDA.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [76]:
from sklearn.metrics import confusion_matrix

In [77]:
conf_mat = confusion_matrix(y_train, LDA.predict(X_train))

In [78]:
pd.DataFrame(conf_mat,
                 columns = ['Predicted 0', 'Predicted 1', 'Predicted 2'],
                 index = ['Actual 0', 'Actual 1', 'Actual 2'])

,Predicted 0,Predicted 1,Predicted 2
Actual 0,2966,194,21
Actual 1,322,1217,2
Actual 2,25,3,17


In [84]:
X=truncate_mfcc(df_bin, avg_depth=40, var_depth=40)
Y = df_bin['critter_num']

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X.copy(), Y,
                                                       shuffle=True,
                                                       random_state=413,
                                                       test_size=.2,
                                                       stratify=Y)

In [86]:
## Make LDA object
LDA = LinearDiscriminantAnalysis()

## Fit the model
LDA.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [87]:
conf_mat = confusion_matrix(y_train, LDA.predict(X_train))

In [88]:
pd.DataFrame(conf_mat,
                 columns = ['Predicted 0', 'Predicted 1'],
                 index = ['Actual 0', 'Actual 1'])

,Predicted 0,Predicted 1
Actual 0,2983,197
Actual 1,326,1215


In [89]:
conf_mat = confusion_matrix(y_test, LDA.predict(X_test))

In [90]:
pd.DataFrame(conf_mat,
                 columns = ['Predicted 0', 'Predicted 1'],
                 index = ['Actual 0', 'Actual 1'])

,Predicted 0,Predicted 1
Actual 0,737,59
Actual 1,96,289
